<a href="https://www.kaggle.com/code/sagorkumarmitra/q-a-chatbot-with-python?scriptVersionId=194048990" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pickle
import numpy as np

with open('/kaggle/input/qa-bot/train_qa.txt','rb') as f1:
    train_data = pickle.load(f1)

with open('/kaggle/input/qa-bot/test_qa.txt','rb') as f2:
    test_data = pickle.load(f2)

In [2]:
type(train_data)

list

In [3]:
type(test_data)

list

In [4]:
len(train_data)

10000

In [5]:
len(test_data)

1000

In [6]:
test_data[0]

(['Mary',
  'got',
  'the',
  'milk',
  'there',
  '.',
  'John',
  'moved',
  'to',
  'the',
  'bedroom',
  '.'],
 ['Is', 'John', 'in', 'the', 'kitchen', '?'],
 'no')

In [7]:
train_data[0]

(['Mary',
  'moved',
  'to',
  'the',
  'bathroom',
  '.',
  'Sandra',
  'journeyed',
  'to',
  'the',
  'bedroom',
  '.'],
 ['Is', 'Sandra', 'in', 'the', 'hallway', '?'],
 'no')

In [8]:
' '.join(train_data[0][0])

'Mary moved to the bathroom . Sandra journeyed to the bedroom .'

In [9]:
all_data = test_data + train_data

In [10]:
vocab = set()

for story,question,answer in all_data:
    vocab = vocab.union(set(story))
    vocab = vocab.union(set(question))
    
vocab.add('no')
vocab.add('yes')

In [11]:
len(vocab)

37

In [12]:
vocab_len = len(vocab) + 1

In [13]:
all_data[0][0]

['Mary',
 'got',
 'the',
 'milk',
 'there',
 '.',
 'John',
 'moved',
 'to',
 'the',
 'bedroom',
 '.']

In [14]:
# Longest Story

all_story_lens = [len(data[0]) for data in all_data]

max_story_len = max(all_story_lens)

In [15]:
max_question_len = max([len(data[1]) for data in all_data])

In [16]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(filters=[])
tokenizer.fit_on_texts(vocab)

2024-08-25 18:17:59.355272: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-25 18:17:59.355429: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-25 18:17:59.506141: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [17]:
tokenizer.word_index

{'mary': 1,
 'no': 2,
 'the': 3,
 'john': 4,
 'yes': 5,
 'kitchen': 6,
 'took': 7,
 'discarded': 8,
 'there': 9,
 'bedroom': 10,
 'picked': 11,
 'went': 12,
 '.': 13,
 '?': 14,
 'is': 15,
 'down': 16,
 'football': 17,
 'left': 18,
 'bathroom': 19,
 'got': 20,
 'garden': 21,
 'sandra': 22,
 'to': 23,
 'journeyed': 24,
 'in': 25,
 'up': 26,
 'hallway': 27,
 'milk': 28,
 'daniel': 29,
 'moved': 30,
 'back': 31,
 'apple': 32,
 'put': 33,
 'grabbed': 34,
 'office': 35,
 'dropped': 36,
 'travelled': 37}

In [18]:
train_story_text = []
train_question_text = []
train_answers = []

for story,question,answer in train_data:
    train_story_text.append(story)
    train_question_text.append(question)
    train_answers.append(answer)

In [19]:
# train_data[0]

In [20]:
# train_story_text[0]

In [21]:
train_story_seq = tokenizer.texts_to_sequences(train_story_text)

In [22]:
len(train_story_seq)

10000

In [23]:
len(tokenizer.word_index)

37

In [24]:
tokenizer.word_index

{'mary': 1,
 'no': 2,
 'the': 3,
 'john': 4,
 'yes': 5,
 'kitchen': 6,
 'took': 7,
 'discarded': 8,
 'there': 9,
 'bedroom': 10,
 'picked': 11,
 'went': 12,
 '.': 13,
 '?': 14,
 'is': 15,
 'down': 16,
 'football': 17,
 'left': 18,
 'bathroom': 19,
 'got': 20,
 'garden': 21,
 'sandra': 22,
 'to': 23,
 'journeyed': 24,
 'in': 25,
 'up': 26,
 'hallway': 27,
 'milk': 28,
 'daniel': 29,
 'moved': 30,
 'back': 31,
 'apple': 32,
 'put': 33,
 'grabbed': 34,
 'office': 35,
 'dropped': 36,
 'travelled': 37}

In [25]:
word_index=tokenizer.word_index

In [26]:
def vectorize_stories(data,word_index=tokenizer.word_index,max_story_len=max_story_len,max_question_len=max_question_len):
    X = [] # Story
    Xq = [] # Questions
    Y = [] # Answer
    
    for story,query,sanswer in data:
        x = [word_index[word.lower()] for word in story]
        xq = [word_index[word.lower()] for word in query]
        
        y = np.zeros(len(word_index)+1)
        
        y[word_index[answer]] = 1
        
        X.append(x)
        Xq.append(xq)
        Y.append(y)
        
    return (pad_sequences(X,maxlen=max_story_len),pad_sequences(Xq,maxlen=max_question_len),np.array(Y))

In [27]:
inputs_train,queries_train,answers_train = vectorize_stories(train_data)

In [28]:
inputs_train

array([[ 0,  0,  0, ...,  3, 10, 13],
       [ 0,  0,  0, ...,  3, 27, 13],
       [ 0,  0,  0, ...,  3, 19, 13],
       ...,
       [ 0,  0,  0, ...,  3, 10, 13],
       [ 0,  0,  0, ..., 28,  9, 13],
       [ 0,  0,  0, ..., 32,  9, 13]], dtype=int32)

In [29]:
queries_train

array([[15, 22, 25,  3, 27, 14],
       [15, 29, 25,  3, 19, 14],
       [15, 29, 25,  3, 35, 14],
       ...,
       [15, 22, 25,  3, 27, 14],
       [15,  1, 25,  3,  6, 14],
       [15,  1, 25,  3, 10, 14]], dtype=int32)

In [30]:
answers_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [31]:
inputs_test,queries_test,answers_test = vectorize_stories(test_data)

In [32]:
inputs_test

array([[ 0,  0,  0, ...,  3, 10, 13],
       [ 0,  0,  0, ...,  3, 21, 13],
       [ 0,  0,  0, ...,  3, 21, 13],
       ...,
       [ 0,  0,  0, ...,  3, 32, 13],
       [ 0,  0,  0, ...,  3, 21, 13],
       [ 0,  0,  0, ..., 32,  9, 13]], dtype=int32)

In [33]:
tokenizer.word_index['yes']

5

In [34]:
tokenizer.word_index['no']

2

In [35]:
sum(answers_test)

array([   0.,    0.,    0.,    0.,    0., 1000.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.])

In [36]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Input,Activation,Dense,Permute,Dropout,add,dot,concatenate,LSTM

# Placeholder shpe = (max_story_len,batch_size)
input_sequence = Input((max_story_len,))
question = Input((max_question_len,))

In [37]:
vocab_size = len(vocab) + 1

# Input Encoder M
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,output_dim=64))
input_encoder_m.add(Dropout(0.3))

# Output


In [38]:
# Input Encoder C
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,output_dim=max_question_len))
input_encoder_c.add(Dropout(0.3))

# Output


In [39]:
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,output_dim=64,input_length=max_question_len))
question_encoder.add(Dropout(0.3))

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [40]:
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

In [41]:
match = dot([input_encoded_m,question_encoded],axes=(2,2))
match = Activation('softmax')(match)

In [42]:
response = add([match,input_encoded_c])
response = Permute((2,1))(response)

In [43]:
answer = concatenate([response,question_encoded])

In [44]:
answer

<KerasTensor shape=(None, 6, 220), dtype=float32, sparse=False, name=keras_tensor_18>

In [45]:
answer = LSTM(32)(answer)
answer = Dropout(0.5)(answer)
answer = Dense(vocab_size)(answer)
answer = Activation('softmax')(answer)

In [46]:
model = Model([input_sequence,question],answer)
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 156)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 6)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 156, 64)   │      2,432 │ input_layer[0][0] │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_2        │ (None, 6, 64)     │      2,432 │ input_layer_1[0]… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot (Dot)           │ (None, 156, 6)    │          0 │ sequential[0][0], │
│                     │                   │            │ sequential_2[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 156, 6)    │          0 │ dot[0][0]         │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_1        │ (None, 156, 6)    │        228 │ input_layer[0][0] │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 156, 6)    │          0 │ activation[0][0], │
│                     │                   │            │ sequential_1[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ permute (Permute)   │ (None, 6, 156)    │          0 │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 6, 220)    │          0 │ permute[0][0],    │
│ (Concatenate)       │                   │            │ sequential_2[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 32)        │     32,384 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 32)        │          0 │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 38)        │      1,254 │ dropout_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 38)        │          0 │ dense[0][0]       │
│ (Activation)        │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 38,730 (151.29 KB)

 Trainable params: 38,730 (151.29 KB)

 Non-trainable params: 0 (0.00 B)